基本的なものは全部乗っているんで、辞書ともいえるかと思います

データ構造の確認  
射影・抽出  
要約統計量  
結合  
グループ化・集約  
欠測値の確認・削除・補完  
重複値の削除  
置換・ソート  
サンプリング  
データ形式の変換  
日時の変換  

In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('for_data_flame').getOrCreate()

In [6]:
print(spark)
df = spark.createDataFrame(
 [(1, 144.5, 5.9, 33, 'M'), (2, 167.2, 5.4, 45, 'M'), (3, 124.1, 5.2, 23, 'F'),
(4, 144.5, 5.9, 33, 'M'), (5, 133.2, 5.7, 54, 'F'), (3, 124.1, 5.2, 23, 'F'),
 (6, 149.3, None, 54, 'M'),],
 ['id', 'weight', 'height', 'age', 'gender'])

射影・抽出

In [7]:
df.select('id','gender').show()

+---+------+
| id|gender|
+---+------+
|  1|     M|
|  2|     M|
|  3|     F|
|  4|     M|
|  5|     F|
|  3|     F|
|  6|     M|
+---+------+



In [8]:
df.filter(df.weight > 145.0).show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  2| 167.2|   5.4| 45|     M|
|  6| 149.3|  null| 54|     M|
+---+------+------+---+------+



In [9]:
df.filter((df.weight > 145.0) | (df.gender == 'M')).show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  1| 144.5|   5.9| 33|     M|
|  2| 167.2|   5.4| 45|     M|
|  4| 144.5|   5.9| 33|     M|
|  6| 149.3|  null| 54|     M|
+---+------+------+---+------+



In [10]:
from pyspark.sql.functions import col
df.filter(~(col("weight") > 145.0) & (col("gender") == 'M')).show()


+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  1| 144.5|   5.9| 33|     M|
|  4| 144.5|   5.9| 33|     M|
+---+------+------+---+------+



In [11]:
df.drop("age").show()

+---+------+------+------+
| id|weight|height|gender|
+---+------+------+------+
|  1| 144.5|   5.9|     M|
|  2| 167.2|   5.4|     M|
|  3| 124.1|   5.2|     F|
|  4| 144.5|   5.9|     M|
|  5| 133.2|   5.7|     F|
|  3| 124.1|   5.2|     F|
|  6| 149.3|  null|     M|
+---+------+------+------+



要約統計量

describe、summaryは、知っているので、省く

In [12]:
#相関係数
df.corr("weight", "height")

-0.18954238554989092

In [13]:
#共分散
df.cov("weight", "height")

-6.160714285714284

結合

In [14]:
df2 = spark.createDataFrame([(1, 'Tokyo',), (2, 'Kyoto',), ], ['id', 'from'])

In [16]:
#join
df2.join(df,df2.id == df.id,'left').show()

+---+-----+---+------+------+---+------+
| id| from| id|weight|height|age|gender|
+---+-----+---+------+------+---+------+
|  1|Tokyo|  1| 144.5|   5.9| 33|     M|
|  2|Kyoto|  2| 167.2|   5.4| 45|     M|
+---+-----+---+------+------+---+------+



In [17]:
#cross join
df.crossJoin(df2.select("from")).select("age", "gender", "from").show()

+---+------+-----+
|age|gender| from|
+---+------+-----+
| 33|     M|Tokyo|
| 33|     M|Kyoto|
| 45|     M|Tokyo|
| 23|     F|Tokyo|
| 45|     M|Kyoto|
| 23|     F|Kyoto|
| 33|     M|Tokyo|
| 54|     F|Tokyo|
| 33|     M|Kyoto|
| 54|     F|Kyoto|
| 23|     F|Tokyo|
| 54|     M|Tokyo|
| 23|     F|Kyoto|
| 54|     M|Kyoto|
+---+------+-----+



?????? ジョインはとても遅いですね

In [19]:
 df3 = spark.createDataFrame([(7, 122.7, 5.1, 36, 'F')],
 ['id', 'weight', 'height', 'age', 'gender'])

In [20]:
df.union(df3).show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  1| 144.5|   5.9| 33|     M|
|  2| 167.2|   5.4| 45|     M|
|  3| 124.1|   5.2| 23|     F|
|  4| 144.5|   5.9| 33|     M|
|  5| 133.2|   5.7| 54|     F|
|  3| 124.1|   5.2| 23|     F|
|  6| 149.3|  null| 54|     M|
|  7| 122.7|   5.1| 36|     F|
+---+------+------+---+------+



In [22]:
df.registerTempTable('df')
df3.registerTempTable('df3')
spark.sql('''select * from df
union all 
select * from df3
'''
).show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  1| 144.5|   5.9| 33|     M|
|  2| 167.2|   5.4| 45|     M|
|  3| 124.1|   5.2| 23|     F|
|  4| 144.5|   5.9| 33|     M|
|  5| 133.2|   5.7| 54|     F|
|  3| 124.1|   5.2| 23|     F|
|  6| 149.3|  null| 54|     M|
|  7| 122.7|   5.1| 36|     F|
+---+------+------+---+------+



groupby

In [23]:
df.select("height").groupBy().mean().collect()

[Row(avg(height)=5.550000000000001)]

In [38]:
df.groupBy('gender').mean().show()

+------+------------------+------------------+-----------------+------------------+
|gender|           avg(id)|       avg(weight)|      avg(height)|          avg(age)|
+------+------------------+------------------+-----------------+------------------+
|     F|3.6666666666666665|127.13333333333333|5.366666666666667|33.333333333333336|
|     M|              3.25|           151.375|5.733333333333334|             41.25|
+------+------------------+------------------+-----------------+------------------+



In [24]:
 df.agg({"age": "mean"}).collect()

[Row(avg(age)=37.857142857142854)]

In [27]:
from pyspark.sql.functions import col

In [28]:
df.filter(col("height").isNull()).show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  6| 149.3|  null| 54|     M|
+---+------+------+---+------+



欠損値

In [32]:
df.filter(col("height").isNull()).show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  6| 149.3|  null| 54|     M|
+---+------+------+---+------+



In [31]:
 df.filter(col("height").isNotNull()).show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  1| 144.5|   5.9| 33|     M|
|  2| 167.2|   5.4| 45|     M|
|  3| 124.1|   5.2| 23|     F|
|  4| 144.5|   5.9| 33|     M|
|  5| 133.2|   5.7| 54|     F|
|  3| 124.1|   5.2| 23|     F|
+---+------+------+---+------+



In [35]:
height_mean = round(df.select("height").groupBy().avg().head()[0], 1)
df.fillna(height_mean).show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  1| 144.5|   5.9| 33|     M|
|  2| 167.2|   5.4| 45|     M|
|  3| 124.1|   5.2| 23|     F|
|  4| 144.5|   5.9| 33|     M|
|  5| 133.2|   5.7| 54|     F|
|  3| 124.1|   5.2| 23|     F|
|  6| 149.3|   5.6| 54|     M|
+---+------+------+---+------+



置換

In [39]:
df.replace(['M', 'F'], ['man', 'women'], 'gender').show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  1| 144.5|   5.9| 33|   man|
|  2| 167.2|   5.4| 45|   man|
|  3| 124.1|   5.2| 23| women|
|  4| 144.5|   5.9| 33|   man|
|  5| 133.2|   5.7| 54| women|
|  3| 124.1|   5.2| 23| women|
|  6| 149.3|  null| 54|   man|
+---+------+------+---+------+



In [43]:
from pyspark.sql.functions import desc
df.orderBy(desc('age')).show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  6| 149.3|  null| 54|     M|
|  5| 133.2|   5.7| 54|     F|
|  2| 167.2|   5.4| 45|     M|
|  1| 144.5|   5.9| 33|     M|
|  4| 144.5|   5.9| 33|     M|
|  3| 124.1|   5.2| 23|     F|
|  3| 124.1|   5.2| 23|     F|
+---+------+------+---+------+



In [44]:
df.sort(df.age.desc()).show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  6| 149.3|  null| 54|     M|
|  5| 133.2|   5.7| 54|     F|
|  2| 167.2|   5.4| 45|     M|
|  4| 144.5|   5.9| 33|     M|
|  1| 144.5|   5.9| 33|     M|
|  3| 124.1|   5.2| 23|     F|
|  3| 124.1|   5.2| 23|     F|
+---+------+------+---+------+



時間

In [47]:
from pyspark.sql.functions import udf, unix_timestamp, to_timestamp

In [46]:
df_ts = spark.createDataFrame([(1, "2018-04-29 16:09:16"), (2, "2018-04-29 18:01:32")], ("id", "dt_str"))


In [48]:
 df_ts.withColumn("ts", unix_timestamp("dt_str")).show()

+---+-------------------+----------+
| id|             dt_str|        ts|
+---+-------------------+----------+
|  1|2018-04-29 16:09:16|1524985756|
|  2|2018-04-29 18:01:32|1524992492|
+---+-------------------+----------+



In [49]:
df_ts.withColumn("dt", to_timestamp("dt_str")).collect()

[Row(id=1, dt_str='2018-04-29 16:09:16', dt=datetime.datetime(2018, 4, 29, 16, 9, 16)),
 Row(id=2, dt_str='2018-04-29 18:01:32', dt=datetime.datetime(2018, 4, 29, 18, 1, 32))]

In [50]:
hour = udf(lambda x: x.hour)

In [51]:
df_ts.withColumn("dt", to_timestamp("dt_str")).withColumn('hour', hour(col('dt'))).show()

+---+-------------------+-------------------+----+
| id|             dt_str|                 dt|hour|
+---+-------------------+-------------------+----+
|  1|2018-04-29 16:09:16|2018-04-29 16:09:16|  16|
|  2|2018-04-29 18:01:32|2018-04-29 18:01:32|  18|
+---+-------------------+-------------------+----+

